## The problem

Given a set of JD products $\{p_1, \dots, p_M \} $, reverse engineer the following function: 

$$JD_{rank} : \{ p_i \} \mapsto \{ 1, 2, \dots, M\}$$

### Not ranked by sales

The data has proven that JD's "by sales"-method of ranking these products is not a simple ordering based on quantity sold.  If it was, we would write the ranking function as follows...

$$JD_{rank}(p_i) = rank(x_{i, q_{30}})$$

where 
- $rank(x_k) = k$ such that $x_1 \geq x_2 \geq \ \dots \ \geq x_K$ and 
- $x_{i, q_{30}}$ is the q30 value of $p_i$

The pictures argue against this concept pretty clearly.  Here's one example:

![example](images/monthly_item_rank_shifts_2.png)

### Rethinking the ranking method

In light of this, we can safely assume the rankings are created by a scoring function.  This is most probably given by

$$score(p_i) = \sum_{j=1}^N a_jx_{i,j} = a_1x_{i,1} + a_2x_{i,2} + a_3x_{i,3} + \dots + a_Nx_{i,N}$$ 

where:

- $x_{i,j} \in \mathbb{Z}$ are some known features about $p_i$ (for example-- allcomments, or the count of products with the same brand/category/fenlei), and
- $a_j \in \mathbb{Q}$ are some unknown parameters, created by JD 

We then would like to search all $a_j$'s such that $rank(score(p_i)) = rank(x_{i,item\_rank})$ for all products $p_i$.  So, the goals are two-fold:

1. Find a solution to the rank problem, and 
2. If a solution exists, minimum values for $a_1,a_2,a_3,a_4$.  For simplicity, we're starting with only 4 $a_j$'s, but we'll be expanding to more later.

In [1]:
from pulp import *

In [2]:
prob = LpProblem("solve for a_i, and minimize the a_i's", LpMinimize)

### 2D case

Let's start with a simple case.  Say we have some $v_1, v_2$ as defined as follows:
$$v_1 =
\left(
\begin{array}{c}
4 \\
8 \\
2 \\
1 \\
\end{array}
\right),
\ v_2 = 
\left(
\begin{array}{c}
2 \\
1 \\
0 \\
0 \\
\end{array}
\right)$$

Later, these $v_j$'s will be replaced with columns from our JD data.  For now, our goal is to see how this kind of problem is formalized for an LP solver, starting with the simpler 2-d case.  Take some linear combination $a_1v_1 + a_2v_2$ of these vectors such that the output is a vector $v_0$ given by

$$a_1v_1 + a_2v_2 = v_0 = 
\left(
\begin{array}{c}
v_0^{(1)} \\
v_0^{(2)} \\
v_0^{(3)} \\
v_0^{(4)} \\
\end{array}
\right)
\ where \ v_0^{(1)} \geq v_0^{(2)} \geq v_0^{(3)} \geq v_0^{(4)} \geq 0$$

In other words, what scalars $a_1,a_2$ can we use on $v_1,v_2$ in order to produce a vector $v_0$ which is ranked in descending order?

In [ ]:
import numpy as np
v1, v2 = np.array([4.,8.,2.,1.]), np.array([2.,1.,0.,0.])

### Employing LP

To make this a LP problem, we'll need:
- an objective function of the form $cx$
- a set of constraints $Ax \geq b$

We could live with any set of $a_1, a_2$ that solves this, but since we need a tangible objective function, let's search for the smallest combination of $a_1, a_2$: $$f(a_1,a_2) = a_1 + a_2$$

Additionally, we'll want a set of $a_1, a_2$ that returns a ranked output vector $v_0$, so our constraints are:

$$ a_1v_1 + a_2v_2 - v_0 = 0 \\ v_0^{(1)} - v_0^{(2)} \geq 0 \\ v_0^{(2)} - v_0^{(3)} \geq 0 \\ v_0^{(3)} - v_0^{(4)} \geq 0$$

In [3]:
a1 = LpVariable("a_1",0,None)
a2 = LpVariable("a_2",0,None)
v01 = LpVariable("v_01",1, None,LpInteger)
v02 = LpVariable("v_02",1, None,LpInteger)
v03 = LpVariable("v_03",1, None,LpInteger)
v04 = LpVariable("v_04",1, None,LpInteger)

prob += a1 + a2, "Find values for x1,x2 which also are the smallest possible values"
prob += v1[0]*a1 + v2[0]*a2 == v01, "first element"
prob += v1[1]*a1 + v2[1]*a2 == v02, "second element"
prob += v1[2]*a1 + v2[2]*a2 == v03, "third element"
prob += v1[3]*a1 + v2[3]*a2 == v04, "fourth element"
prob += v01 - v02 >= 0, "v01 > v02"
prob += v02 - v03 >= 0, "v02 > v03"
prob += v03 - v04 >= 0, "v03 > v04"

In [5]:
prob.solve()

1

In [6]:
for v in prob.variables():
    print v.name, "\t=", v.varValue

a_1 	= 1.0
a_2 	= 4.0
v_01 	= 12.0
v_02 	= 12.0
v_03 	= 2.0
v_04 	= 1.0


Validate results by plugging values back into equation and evaluating the function

In [7]:
print "Function output at solved ai's:",prob.variablesDict()["a_1"].varValue*v1+prob.variablesDict()["a_2"].varValue*v2

Function output at solved ai's: [ 12.  12.   2.   1.]


### 4D case

Now we'll try adding two vectors $v_3,v_4$ to the mix, to simulate the problem we are trying to solve.

$$v_1 =
\left(
\begin{array}{c}
4 \\
8 \\
2 \\
1 \\
\end{array}
\right),
\ v_2 = 
\left(
\begin{array}{c}
2 \\
1 \\
0 \\
0 \\
\end{array}
\right),
\ v_3 = 
\left(
\begin{array}{c}
1 \\
2 \\
1 \\
2 \\
\end{array}
\right), 
\ v_4 = 
\left(
\begin{array}{c}
5 \\
1 \\
1 \\
2 \\
\end{array}
\right)
$$

In [14]:
v1, v2 = np.array([4,8,2,1]), np.array([2,1,0,0])
v3, v4 = np.array([1,2,1,7]), np.array([5,1,1,2])

In [15]:
a1 = LpVariable("a_1",1,None,LpInteger)
a2 = LpVariable("a_2",1,None,LpInteger)
a3 = LpVariable("a_3",1,None,LpInteger)
a4 = LpVariable("a_4",1,None,LpInteger)
v01 = LpVariable("v_01",1, None,LpInteger)
v02 = LpVariable("v_02",1, None,LpInteger)
v03 = LpVariable("v_03",1, None,LpInteger)
v04 = LpVariable("v_04",1, None,LpInteger)

prob4d = LpProblem("solve for a_i, and minimize the a_i's -- 4D case", LpMinimize)
prob4d += a1 + a2 + a3 + a4
prob4d += v1[0]*a1 + v2[0]*a2 + v3[0]*a3 + v4[0]*a4 == v01, "first element"
prob4d += v1[1]*a1 + v2[1]*a2 + v3[1]*a3 + v4[1]*a4 == v02, "second element"
prob4d += v1[2]*a1 + v2[2]*a2 + v3[2]*a3 + v4[2]*a4 == v03, "third element"
prob4d += v1[3]*a1 + v2[3]*a2 + v3[3]*a3 + v4[3]*a4 == v04, "fourth element"
prob4d += v01 - v02 >= 0, "v01 > v02"
prob4d += v02 - v03 >= 0, "v02 > v03"
prob4d += v03 - v04 >= 0, "v03 > v04"
prob4d.solve()

1

In [16]:
for v in prob4d.variables():
    print v.name, "\t=", v.varValue

a_1 	= 7.0
a_2 	= 25.0
a_3 	= 1.0
a_4 	= 1.0
v_01 	= 84.0
v_02 	= 84.0
v_03 	= 16.0
v_04 	= 16.0


Now, change the value of $v_2^{(4)}$=5, so now the vectors are...


$$v_1 =
\left(
\begin{array}{c}
4 \\
8 \\
2 \\
1 \\
\end{array}
\right),
\ v_2 = 
\left(
\begin{array}{c}
2 \\
1 \\
0 \\
\color{orangered} 5 \\
\end{array}
\right),
\ v_3 = 
\left(
\begin{array}{c}
1 \\
2 \\
1 \\
2 \\
\end{array}
\right), 
\ v_4 = 
\left(
\begin{array}{c}
5 \\
1 \\
1 \\
2 \\
\end{array}
\right)
$$

In [18]:
v2 = np.array([2,1,0,5])

In [19]:
prob4d2 = LpProblem("solve for a_i, and minimize the a_i's -- 4D case", LpMinimize)
prob4d2 += a1 + a2 + a3 + a4
prob4d2 += v1[0]*a1 + v2[0]*a2 + v3[0]*a3 + v4[0]*a4 == v01, "first element"
prob4d2 += v1[1]*a1 + v2[1]*a2 + v3[1]*a3 + v4[1]*a4 == v02, "second element"
prob4d2 += v1[2]*a1 + v2[2]*a2 + v3[2]*a3 + v4[2]*a4 == v03, "third element"
prob4d2 += v1[3]*a1 + v2[3]*a2 + v3[3]*a3 + v4[3]*a4 == v04, "fourth element"
prob4d2 += v01 - v02 >= 0, "v01 > v02"
prob4d2 += v02 - v03 >= 0, "v02 > v03"
prob4d2 += v03 - v04 >= 0, "v03 > v04"
prob4d2.solve()

-1

In [22]:
LpStatus[-1]

'Infeasible'

## Moving to Compass data

Notice that not every set of 4 vectors will have a solution.  Indeed, as we progress into Compass data, solutions will likely be very difficult to find for large $v_i$'s...

![too much data](images/monthly_item_rank_shifts_4.png)  

In the same vein, we won't want $v_i$'s which are too small....

![not enough data](images/monthly_item_rank_shifts_5.png)

...because we'll get too many solutions, most of which won't add a lot of insight.

We'll be hoping for an amount of data that describes everything accurately, but also isn't so dense that it will be impossible to solve...

![just right](images/monthly_item_rank_shifts_1.png)

This is a case where the cleanliness and completeness of the data is vitally important.  Any rows/columns with missing values or unclean data will need to be removed as a pre-processing step.

## Formalizing the problem in Compass terms

We will formalize the 3 __problem-types__ to consider for compass data: product-level, page-level, and pageset-level. These will be explained in more detail below, but it's better to get a picture of these problem-types first, noticing that each one allows for a more relaxed ranking criterion than the last:

![just right](images/sample_solutions.png)

M is the length of the solution vector $v_0$ discovered for some problem-type.  Alternatively, it's the number of products for which we were able to find a scoring function which ranked them according to the problem-type.  It's important to remember that when we are solving for 70+ products, and 40+ attributes ($a_i$'s), it is likely that a solution will not be found until we subsample that set of products to around 20-40 products.  The subsampling process will almost always contain different permutations of products $p_i$, so M will vary over some probability distribution.  Because page-level is more relaxed than product-level, we'll expect larger values of M for page-level than product-level.  Similarly, M for pageset-level should be larger than both.


### Product-level

This will be a matter of replacing the $v_j$'s from above with columns from the compass database ($x_j$).  Remember, the scoring function we are interested is given by $$score(p_i) = \sum_{j=1}^N a_jx_{i,j}$$

where $N$ is the total number of columns.  It's given that 

$$JD_{rank}(p_i) = rank(x_{i,item\_rank})$$

So we are looking for $a_j$ such that $$rank(x_{i,item\_rank}) = rank(score(p_i)) = rank(\sum_{j=1}^N a_jx_{i,j}) \ \text{for all} \  i \in \{ 1, \dots, M\}$$

So in the context of LP, this can be written as follows:

$$\text{Objective function: } \sum_{j=1}^N a_j \\ \text{Constraints:} \sum_j a_jx_j - v_0 = 0 \text{ where } x_j \text{ is the } j^{th} \text{column} \\ v_0^{(1)} \geq v_0^{(2)} \geq \dots \geq v_0^{(M)}$$

This problem is created by the rankLP.py module's _rankByProduct()_ method.  All one needs to do is give the method an input dataframe which is sorted by item_rank.

In [3]:
import rankLP as rlp
from examples.df929 import df

probCompassProd = rlp.forceSolution(df, by="product")
rlp.viewAs(probCompassProd,.2)
rlp.viewVs(probCompassProd)

a__DOVE 	= 2.0927435
a__HAZELINE 	= 10.0
a__agg07 	= 8.7671777
a__agg13 	= 6.6528283
a__c45 	= 0.55728258
a__currentprice 	= 4.5087417
a__featagg043 	= 3.3110228
a__featagg073 	= 10.0
a__hasAttr00 	= 10.0
a__is_in_stock 	= 10.0
v_0_0004 	= 99.310178
v_0_0039 	= 40.839112
v_0_0041 	= 39.974293
v_0_0048 	= 33.144852
v_0_0056 	= 33.144852
v_0_0075 	= 33.144852
v_0_0083 	= 27.742874
v_0_0085 	= 27.259292
v_0_0090 	= 26.167797
v_0_0106 	= 25.159586
v_0_0127 	= 25.159586
v_0_0129 	= 25.159586
v_0_0158 	= 25.159586
v_0_0189 	= 22.062178
v_0_0410 	= 21.267164
v_0_0414 	= 21.267164
v_0_0426 	= 21.267164
v_0_0457 	= 21.267164
v_0_0461 	= 19.551999
v_0_0730 	= 19.551999
v_0_0735 	= 19.551999
v_0_1322 	= 19.551999


### page-level

Assume we are only interested in whether an item is ranked correctly according to page -- meaning that the exact order of the score doesn't matter, as long as products do not step over page boundaries.  Let's call the set of pages $\{ \beta_1, \dots, \beta_P \}$.  In this case, we'll have to create another feature $x_{page}$ which is created by the following function:

$$pageranker(x_{i,item\_rank}) = 
\left\{
\begin{array}{cc}
      \beta_1 & 0 \leq x_{i,item\_rank} \leq 60 \\
      \beta_2 & 60 \leq x_{i,item\_rank} \leq 120 \\
      \beta_3 & 120 \leq x_{i,item\_rank} \leq 180 \\
      \dots & \dots\\
\end{array} 
\right\}$$

After this is done and ranked, we'll be able to organize products $p_i$ thusly:

$$
\begin{array}{rr}
\{ p_1, \dots, p_{b_1} \} = & \beta_1 \\ 
\{ p_{({b_1}+1)}, \dots, p_{b_2} \} = & \beta_2 \\ 
\{ p_{({b_2}+1)}, \dots, p_{b_3} \} = & \beta_3 \\ 
\dots \\  
\{ p_{(b_{P-1}+1)}, \dots, p_{b_P} \} = & \beta_P \\
\end{array}$$

Putting that into an LP context:

$$
\text{Objective function:} \sum_j a_j \\
\text{Constraints:} \sum_j a_jx_j = v_0 \\
\ \beta_1 \geq \beta_2 \geq \dots \geq \beta_P \implies \{ v_0^{(1)}, \dots, v_0^{(b_1)} \} \geq \{ v_0^{(b_1+1)}, \dots, v_0^{(b_2)} \} \geq \dots \geq \{ v_0^{(b_{P-1}+1)}, \dots, v_0^{(b_P)} \} 
$$



This problem is created by the rankLP.py module's _rankByPage()_ method.

### page-set-level

We could also be even more general, and allow for a product to appear on a set of pages.  For example, a good starting point would be:

- $\psi_1 = \{ \beta_1 \}$
- $\psi_2 = \{ \beta_2, \beta_3 \}$
- $\psi_3 = \{ \beta_4, \beta_5, \beta_6, \beta_7 \}$
- $\psi_4 = \{ \beta_8 , \beta_9, \dots, \beta_P \}$

...which is not a hard thing to translate into another LP context:

$$
\text{Objective function:} \sum_j a_j \\
\text{Constraints:} \sum_j a_jx_j = v_0  \\
\ \psi_1 \geq \psi_2 \geq \psi_3 \geq \psi_4 \implies \{ v_0^{(1)}, \dots, v_0^{(b_1)} \} \geq \{ v_0^{(b_1+1)}, \dots, v_0^{(b_3)} \} \geq \{ v_0^{(b_3+1)}, \dots, v_0^{(b_7)} \geq \{ v_0^{(b_7+1)}, \dots, v_0^{(b_P)} \} 
$$

This problem is created by the rankLP.py module's _rankByPageset()_ method.

### rankLP.py module -- LP methods

#### rankByProduct(), rankByPage(), rankByPageset()
These simply create the problem, given an input dataframe.  They do not solve the problem.  The input dataframe must be sorted by and indexed by the item_rank.  Other notes:
- rankByPage()/rankByPageset()-- the page number is required.  This can be generated by rankLP's addPage() method.
- rankByPageset()-- custom pagesets can be optionally defined at input.  Defaults to the one defined above.  

Once the problem is created, it is not hard to solve -- simply invoke pulp's solve() method.  Whether there is a solution is not certain.  This contingency is handled by the forceSolution() method.

#### forceSolution()
forceSolution() tries to find a solution, and if it can't, continually samples the input dataframe at random, reducing the sample size by 1 on each iteration, until a solution can be found, at which point it stops and returns the details.